In [ ]:
import os
import numpy as np
import pandas as pd
import random
import seaborn as sns

import datetime as datetime
import matplotlib.dates as dates
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from contextlib import contextmanager
from time import time
from tqdm import tqdm
import lightgbm as lgbm

from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

In [ ]:
data0 = pd.read_csv("../input/covid19-indonesia/covid_19_indonesia_time_series_all.csv")
data0[-5:]

In [ ]:
data0['Location'].unique()

In [ ]:
import datetime
from datetime import date
date2=[]
for item in data0['Date']:
    item2=item.split('/')
    month=int(item2[0])
    day=int(item2[1])
    year=int(item2[2])
    date2+=[datetime.date(year,month,day)]
data0['Date2']=date2
data0

In [ ]:
data1=data0.groupby('Date2',as_index=False).sum()
data1

# New Cases rolling 7 days

In [ ]:
data2=data1[['Date2','New Cases']]
data2

In [ ]:
data2['positives mean 7-day']=data2['New Cases'].rolling(window=7).mean()
data2['positives max 7-day']=data2['New Cases'].rolling(window=7).max()
data2['positives min 7-day']=data2['New Cases'].rolling(window=7).min()
data2[-5:]

In [ ]:
fig=make_subplots(specs=[[{"secondary_y":False}]])
fig.add_trace(go.Scatter(x=data2['Date2'],y=data2['positives mean 7-day'],name='positives mean 7-day'),secondary_y=False,)
fig.add_trace(go.Scatter(x=data2['Date2'],y=data2['positives max 7-day'],name='positives max 7-day'),secondary_y=False,)
fig.add_trace(go.Scatter(x=data2['Date2'],y=data2['positives min 7-day'],name='positives min 7-day'),secondary_y=False,)
fig.update_layout(autosize=False,width=700,height=500,title_text="New Cases (rolling 7-day) in Indonesia")
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Cases",secondary_y=False)
fig.show()

In [ ]:
col0=data2.columns.to_list()
col1=col0+['pm-7','slope']
data3=pd.DataFrame(columns=col1)
data3[col0]=data2
data3

In [ ]:
n=len(data2)
for  i in range(n):    
    pmi=data3['positives mean 7-day'][i]
    data3.loc[i+7,'pm-7']=pmi

In [ ]:
### Slope value is defined as the following formula. 
# 'pm-7' means 'positives mean 7-day' value 7 days before.

data3['slope']=(data3['positives mean 7-day']-data3['pm-7'])/7
data4=data3[['Date2','New Cases','positives mean 7-day','pm-7','slope']]
data5=data4[510:-7]

print(data5[['Date2','slope']][0:14])
print()
print(data5[['Date2','slope']][-14:])

### A slope value is a sensitive benchmark for an occurence of infectious explosion in the near future.

In [ ]:
fig=make_subplots(specs=[[{"secondary_y":True}]])
fig.add_trace(go.Scatter(x=data5['Date2'],y=data5['slope'],name='slope'),secondary_y=False,)
fig.add_trace(go.Scatter(x=data5['Date2'],y=data5['positives mean 7-day'],name='New Cases'),secondary_y='positives mean 7-day')
fig.update_layout(autosize=False,width=700,height=500,title_text="Slope (mean daily increase) of New Cases in Indonesia")
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Slope",secondary_y=False)
fig.update_yaxes(title_text="New Cases",secondary_y=True)
fig.show()